# TSA Chapter 5: News Impact Curve

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_nic/TSA_ch5_nic.ipynb)

News Impact Curves from estimated GARCH and GJR-GARCH models on S&P 500.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
import yfinance as yf

# Download S&P 500 data
sp = yf.download('^GSPC', start='2000-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
sp500_returns = (sp['Close'].pct_change() * 100).dropna()
sp500_returns = pd.Series(sp500_returns.values, index=sp500_returns.index, name='returns')
print(f"S&P 500: {len(sp500_returns)} observations")
from arch import arch_model

# Fit GARCH models on S&P 500
m_g = arch_model(sp500_returns, vol='Garch', p=1, q=1, dist='normal')
r_g = m_g.fit(disp='off')
m_j = arch_model(sp500_returns, vol='Garch', p=1, o=1, q=1, dist='normal')
r_j = m_j.fit(disp='off')
m_e = arch_model(sp500_returns, vol='EGARCH', p=1, q=1, dist='normal')
r_e = m_e.fit(disp='off')
models = {'garch': r_g, 'gjr': r_j, 'egarch': r_e}
print("GARCH models fitted successfully")

In [ ]:
# News Impact Curve (estimated from real data)
r_g = models['garch']; r_j = models['gjr']
omega_g = r_g.params['omega']; alpha_g = r_g.params['alpha[1]']; beta_g = r_g.params['beta[1]']
omega_j = r_j.params['omega']; alpha_j = r_j.params['alpha[1]']
gamma_j = r_j.params['gamma[1]']; beta_j = r_j.params['beta[1]']

sigma2_prev_g = omega_g / max(1 - alpha_g - beta_g, 0.01)
sigma2_prev_j = omega_j / max(1 - alpha_j - gamma_j/2 - beta_j, 0.01)

eps_range = np.linspace(-5, 5, 200)
nic_garch = omega_g + alpha_g * eps_range**2 + beta_g * sigma2_prev_g
indicator = (eps_range < 0).astype(float)
nic_gjr = omega_j + alpha_j * eps_range**2 + gamma_j * eps_range**2 * indicator + beta_j * sigma2_prev_j

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(eps_range, np.sqrt(nic_garch), color=BLUE, lw=2.5,
        label=f'GARCH(1,1): $\\alpha$={alpha_g:.3f}, $\\beta$={beta_g:.3f}')
ax.plot(eps_range, np.sqrt(nic_gjr), color=RED, lw=2.5,
        label=f'GJR-GARCH: $\\alpha$={alpha_j:.3f}, $\\gamma$={gamma_j:.3f}, $\\beta$={beta_j:.3f}')
ax.axvline(0, color=GRAY, ls=':', lw=1)
ax.set_xlabel('Shock $\\varepsilon_{t-1}$ (%)')
ax.set_ylabel('Conditional Volatility $\\sigma_t$ (%)')
ax.text(0.02, 0.98, 'Negative shocks have greater\nimpact on volatility',
        transform=ax.transAxes, fontsize=9, va='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
legend_bottom(ax, ncol=2, y=-0.18)
fig.tight_layout(); fig.subplots_adjust(bottom=0.20)
save_chart(fig, 'garch_news_impact_curve'); plt.show()